In [1]:
import time
from math import exp, pow, ceil, pi
from typing import Callable

import numpy as np
import SALib
from SALib.sample import saltelli
from SALib.analyze import sobol
from numba import njit
from sympy import N

In [35]:
import warnings
warnings.filterwarnings("ignore")

# 1. Naive

In [36]:
def model(x: np.ndarray):
    N = len(x)
    sum_ = 0
    for i in range(N):
        sum_ = sum_ + x[i] ** (N - i)
    return sum_

In [37]:
def full_template(model: Callable, problem: dict, n: int, name: str) -> float:
    full_time = 0
    print(name)
    start = time.time()
    param_values = saltelli.sample(problem, n)
    end = time.time()
    full_time += (end - start)
    print("Samples generation took {} seconds".format(end - start))

    """3. Evaluate the model using the generated inputs, saving the model outputs."""
    start = time.time()

    Y = np.zeros([param_values.shape[0]])
    for i, X in enumerate(param_values):
        Y[i] = model(X)
    Si = sobol.analyze(problem, Y)

    end = time.time()
    full_time += (end - start)
    print("Model evaluation took {} seconds".format(end - start))
    print("Si: {}".format(Si['S1']))
    print(f"Full time: {full_time}")
    print("----------")
    return full_time

# Problem formulation

In [38]:
"""1. Determine the model inputs (parameters) and their sample range
"""
N_VARS = 6
problem = {
    'num_vars': N_VARS,
    'names': [f"x{i + 1}" for i in range(N_VARS)],
    'bounds': [[-2.0, 2.0] for _ in range(N_VARS)]
}

"""2. Run the sample function to generate the model inputs."""
n = 100000

In [39]:
t1 = full_template(model, problem, n, "without acceleration")

without acceleration
Samples generation took 8.28935980796814 seconds
Model evaluation took 16.468003034591675 seconds
Si: [0.65262683 0.26246096 0.05131471 0.02577939 0.00400679 0.00375024]
Full time: 24.757362842559814
----------


# 2.1. Numba

In [40]:
@njit
def model_njit(x: np.ndarray):
    N = len(x)
    sum_ = 0
    for i in range(N):
        sum_ = sum_ + x[i] ** (N - i)
    return sum_

In [41]:
t2 = full_template(model_njit, problem, n, "with numba")

with numba
Samples generation took 8.51076602935791 seconds
Model evaluation took 13.439954996109009 seconds
Si: [0.65262683 0.26246096 0.05131471 0.02577939 0.00400679 0.00375024]
Full time: 21.95072102546692
----------


Кажется, ускорения не особо получилось. Попробуем использовать list comprehension

# 2.2. List comprehension

In [42]:
def model_list(x: np.ndarray):
    N = len(x)
    return sum([each ** (N - i) for i, each in enumerate(x)])

In [43]:
t3 = full_template(model_list, problem, n, "with list comprehension")

with list comprehension
Samples generation took 8.170188665390015 seconds
Model evaluation took 17.127964973449707 seconds
Si: [0.65262683 0.26246096 0.05131471 0.02577939 0.00400679 0.00375024]
Full time: 25.29815363883972
----------


Так почему-то не лучше, чем naive способ.

# 2.3. Pybind11

Написал **model_cpp** в **module.cpp**, но почему-то на М1 возникли какие-то проблемы с Pybind11, так и не получилось завести, но файл все равно приложил рядом. Компилировал так же, как на официальном сайте: </br>
`g++ -O3 -Wall -shared -std=c++11 -fPIC $(/Users/photosartd/opt/anaconda3/bin/python -m pybind11 --includes) module.cpp -o module.so`

# 3. Paralleling

Попробуем разбить проблему на **N_VARS** проблем c помощью **multiprocessing**:

In [44]:
from multiprocessing import Pool

In [45]:
problems_parellel = [{
    "num_vars": 1, "names": [f"x{i + 1}"], "bounds": [[-2.0, 2.0]]
} for i in range(N_VARS)]

In [64]:
COEF = 3.5

Коэффициент выбран по формуле из asltelli.sample для того, чтобы при проблеме с размерностью 1 было сгенерировано такое же число точек (1400000), как и с размерностью 6

In [65]:
start = time.time()
with Pool(6) as p:
    param_values_list = np.hstack(
            p.starmap(
            saltelli.sample,
            zip(problems_parellel,
                [int(n * COEF) for _ in range(N_VARS)]
               )
        )
    )
end = time.time()
print("Samples generation took {} seconds".format(end - start))

Samples generation took 7.098089933395386 seconds


/Users/photosartd/opt/anaconda3/lib/python3.8/site-packages/SALib/sample/saltelli.py:109: UserWarning: 
        Convergence properties of the Sobol' sequence is only valid if
        `N` (350000) is equal to `2^n`.
        
  warnings.warn(msg)
/Users/photosartd/opt/anaconda3/lib/python3.8/site-packages/SALib/sample/saltelli.py:109: UserWarning: 
        Convergence properties of the Sobol' sequence is only valid if
        `N` (350000) is equal to `2^n`.
        
  warnings.warn(msg)
/Users/photosartd/opt/anaconda3/lib/python3.8/site-packages/SALib/sample/saltelli.py:109: UserWarning: 
        Convergence properties of the Sobol' sequence is only valid if
        `N` (350000) is equal to `2^n`.
        
  warnings.warn(msg)
/Users/photosartd/opt/anaconda3/lib/python3.8/site-packages/SALib/sample/saltelli.py:109: UserWarning: 
        Convergence properties of the Sobol' sequence is only valid if
        `N` (350000) is equal to `2^n`.
        
  warnings.warn(msg)
/Users/photosartd/op

In [66]:
param_values_list.shape[0] == 1400000

True

Кажется, с помощью мультипроцессинга мы смогли немного распараллелить набрасывание точек.

In [71]:
start = time.time()

print(param_values_list.shape[0])
Y = np.zeros([param_values_list.shape[0]])
for i, X in enumerate(param_values_list):
    Y[i] = model_njit(X)
Si = sobol.analyze(problem, Y)

end = time.time()
print("Model evaluation took {} seconds".format(end - start))

1400000
Model evaluation took 13.68556809425354 seconds


Пока мы смогли сократить время с 24 до 20 секунд, но при бОльшем числе переменных разница будет увеличиваться